In [1]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.vis_utils import plot_model
from keras.applications.mobilenet_v2 import decode_predictions
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow
from time import time

import pandas

import pickle as pk

from keras.wrappers.scikit_learn import KerasClassifier

import numpy as np

seed = 7

Using TensorFlow backend.


## Evaluate Batch size with RMSprop.
#### First, evaluate each size independently (restart kernel each time without clearing output).
#### Then, to the, all sequential to get approximation of the weights really fast and improve accuracy sequentially

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4997893463339097049
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11280557671
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12220713136331592485
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [3]:
dir(keras.applications.mobilenet_v2)

['MobileNetV2',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'absolute_import',
 'decode_predictions',
 'division',
 'keras_modules_injection',
 'mobilenet_v2',
 'preprocess_input',
 'print_function']

In [4]:
np.random.seed(seed)

In [5]:
data_dir = '/data/oxford102/train'
img_width, img_height = 256, 256
batch_size = 32
epochs = 100
nr_categories = 102

In [6]:
img_width, img_height = 256, 256
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
nb_train_samples = 4604
nb_validation_samples = 1094 

In [7]:
input_processor = applications.mobilenet_v2.preprocess_input

In [8]:
train_val_datagen = ImageDataGenerator(preprocessing_function=input_processor, 
                                           validation_split=0.2)

train_val_datagen_aug = ImageDataGenerator(
        #rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=input_processor,
        validation_split=0.2)

In [9]:

    
train_generator = train_val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

validation_generator = train_val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.


In [10]:
#from keras import regularizers

k.set_learning_phase(0)

network_name = "inception_resnet_v2"
img_width, img_height = (256, 256)
if network_name == "vgg16":
    base_model = keras.applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
elif network_name == "vgg19":
    base_model = keras.applications.vgg19.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
elif network_name == "inception_resnet_v2":
    base_model = keras.applications.inception_resnet_v2.InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
elif network_name == "mobilenet_v2":
    base_model = keras.applications.mobilenet_v2.MobileNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

else:
    raise Exception("check your network name")

for layer in base_model.layers[:]:
    layer.trainable = False

    #Adding custom Layers 
k.set_learning_phase(1)
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu", 
          #kernel_regularizer=regularizers.l2(0.01),
         #       activity_regularizer=regularizers.l1(0.001)
         )(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x, training=True)
#x = Dense(102, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

_model = Model(input = base_model.input, output = predictions)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:35: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [11]:
_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 127, 127, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 127, 127, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [ ]:
# _model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [ ]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': model_name,
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 512,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

In [ ]:
print(1)

In [ ]:
model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094

params = {'network_name': model_name,
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 256,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

In [12]:
model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
35/35 [==============================] - 30s 861ms/step - loss: 3.5846 - acc: 0.2348 - val_loss: 3.0738 - val_acc: 0.3203

Epoch 00001: val_acc improved from -inf to 0.32031, saving model to /data/oxford102/experiments/1544313888.896595/mobilenet_v2_1544313888.896595.h5
Epoch 2/100
35/35 [==============================] - 24s 679ms/step - loss: 2.8652 - acc: 0.3402 - val_loss: 2.7228 - val_acc: 0.3711

Epoch 00002: val_acc improved from 0.32031 to 0.37109, saving model to /data/oxford102/experiments/1544313888.896595/mobilenet_v2_1544313888.896595.h5
Epoch 3/100
35/35 [==============================] - 24s 684ms/step - loss: 2.5635 - acc: 0.4232 - val_loss: 2.3986 - val_acc: 0.4766

Epoch 00003: val_acc improved from 0.37109 to 0.47656, saving model to /data/oxford102/experiments/1544313888.896595/mobilenet_v2_1544313888.896595.h5
Epoch 4/100
35/35 [==============================] - 24s 683ms/step - loss: 2.4074 - acc: 0.4580 - val_loss: 2.2437 - val_acc: 0.5156

Epoch 0000

In [12]:
model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
71/71 [==============================] - 55s 769ms/step - loss: 3.1814 - acc: 0.3072 - val_loss: 2.6400 - val_acc: 0.4320

Epoch 00001: val_acc improved from -inf to 0.43199, saving model to /data/oxford102/experiments/1544321516.4379833/mobilenet_v2_1544321516.4379833.h5
Epoch 2/100
71/71 [==============================] - 49s 685ms/step - loss: 2.4163 - acc: 0.4626 - val_loss: 2.2147 - val_acc: 0.5055

Epoch 00002: val_acc improved from 0.43199 to 0.50551, saving model to /data/oxford102/experiments/1544321516.4379833/mobilenet_v2_1544321516.4379833.h5
Epoch 3/100
71/71 [==============================] - 50s 704ms/step - loss: 1.6797 - acc: 0.6342 - val_loss: 1.9133 - val_acc: 0.5695

Epoch 00003: val_acc improved from 0.50551 to 0.56950, saving model to /data/oxford102/experiments/1544321516.4379833/mobilenet_v2_1544321516.4379833.h5
Epoch 4/100
71/71 [==============================] - 49s 688ms/step - loss: 1.5051 - acc: 0.6774 - val_loss: 1.6431 - val_acc: 0.6121

Epoc

In [12]:
model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
143/143 [==============================] - 104s 728ms/step - loss: 2.8257 - acc: 0.3722 - val_loss: 2.2077 - val_acc: 0.5000

Epoch 00001: val_acc improved from -inf to 0.50000, saving model to /data/oxford102/experiments/1544322364.962447/mobilenet_v2_1544322364.962447.h5
Epoch 2/100
143/143 [==============================] - 99s 692ms/step - loss: 1.6082 - acc: 0.6630 - val_loss: 1.7285 - val_acc: 0.6036

Epoch 00002: val_acc improved from 0.50000 to 0.60358, saving model to /data/oxford102/experiments/1544322364.962447/mobilenet_v2_1544322364.962447.h5
Epoch 3/100
143/143 [==============================] - 98s 684ms/step - loss: 1.0723 - acc: 0.7830 - val_loss: 1.4895 - val_acc: 0.6375

Epoch 00003: val_acc improved from 0.60358 to 0.63748, saving model to /data/oxford102/experiments/1544322364.962447/mobilenet_v2_1544322364.962447.h5
Epoch 4/100
143/143 [==============================] - 98s 683ms/step - loss: 0.7257 - acc: 0.8654 - val_loss: 1.2789 - val_acc: 0.6836

E

In [11]:
model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'mobilenet_v2',
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 16,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
287/287 [==============================] - 204s 711ms/step - loss: 2.1876 - acc: 0.5239 - val_loss: 1.6708 - val_acc: 0.6093

Epoch 00001: val_acc improved from -inf to 0.60930, saving model to /data/oxford102/experiments/1544324718.4162903/mobilenet_v2_1544324718.4162903.h5
Epoch 2/100
287/287 [==============================] - 196s 682ms/step - loss: 0.8537 - acc: 0.8272 - val_loss: 1.3012 - val_acc: 0.6784

Epoch 00002: val_acc improved from 0.60930 to 0.67844, saving model to /data/oxford102/experiments/1544324718.4162903/mobilenet_v2_1544324718.4162903.h5
Epoch 3/100
287/287 [==============================] - 196s 683ms/step - loss: 0.4377 - acc: 0.9240 - val_loss: 1.1310 - val_acc: 0.7269

Epoch 00003: val_acc improved from 0.67844 to 0.72693, saving model to /data/oxford102/experiments/1544324718.4162903/mobilenet_v2_1544324718.4162903.h5
Epoch 4/100
287/287 [==============================] - 196s 683ms/step - loss: 0.2630 - acc: 0.9578 - val_loss: 1.1568 - val_acc: 

In [12]:
model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'mobilenet_v2',
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
575/575 [==============================] - 403s 701ms/step - loss: 1.5411 - acc: 0.6723 - val_loss: 1.3399 - val_acc: 0.6692

Epoch 00001: val_acc improved from -inf to 0.66916, saving model to /data/oxford102/experiments/1544399249.5668316/mobilenet_v2_1544399249.5668316.h5
Epoch 2/100
575/575 [==============================] - 393s 683ms/step - loss: 0.3679 - acc: 0.9373 - val_loss: 1.1550 - val_acc: 0.7090

Epoch 00002: val_acc improved from 0.66916 to 0.70904, saving model to /data/oxford102/experiments/1544399249.5668316/mobilenet_v2_1544399249.5668316.h5
Epoch 3/100
575/575 [==============================] - 392s 682ms/step - loss: 0.1783 - acc: 0.9726 - val_loss: 1.1863 - val_acc: 0.7048

Epoch 00003: val_acc did not improve from 0.70904
Epoch 4/100
575/575 [==============================] - 392s 682ms/step - loss: 0.1210 - acc: 0.9810 - val_loss: 1.1200 - val_acc: 0.7149

Epoch 00004: val_acc improved from 0.70904 to 0.71492, saving model to /data/oxford102/experime

# RUN BATCHE SIZES SEQUENTIAL (no restart of kernel)

In [13]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': model_name,
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 512,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
8/8 [==============================] - 9s 1s/step - loss: 0.1649 - acc: 0.9844 - val_loss: 1.1774 - val_acc: 0.7344

Epoch 00001: val_acc improved from -inf to 0.73438, saving model to /data/oxford102/experiments/1544329528.7978303/mobilenet_v2_1544329528.7978303.h5
Epoch 2/100
8/8 [==============================] - 6s 698ms/step - loss: 0.2239 - acc: 0.9727 - val_loss: 1.2865 - val_acc: 0.6875

Epoch 00002: val_acc did not improve from 0.73438
Epoch 3/100
8/8 [==============================] - 5s 683ms/step - loss: 0.2118 - acc: 0.9688 - val_loss: 1.0640 - val_acc: 0.7812

Epoch 00003: val_acc improved from 0.73438 to 0.78125, saving model to /data/oxford102/experiments/1544329528.7978303/mobilenet_v2_1544329528.7978303.h5
Epoch 4/100
8/8 [==============================] - 6s 688ms/step - loss: 0.1862 - acc: 0.9766 - val_loss: 1.4649 - val_acc: 0.6094

Epoch 00004: val_acc did not improve from 0.78125
Epoch 5/100
8/8 [==============================] - 5s 684ms/step - loss:

In [14]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': model_name,
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 256,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
17/17 [==============================] - 17s 982ms/step - loss: 0.2301 - acc: 0.9651 - val_loss: 1.3921 - val_acc: 0.6250

Epoch 00001: val_acc improved from -inf to 0.62500, saving model to /data/oxford102/experiments/1544329584.4107325/mobilenet_v2_1544329584.4107325.h5
Epoch 2/100
17/17 [==============================] - 11s 661ms/step - loss: 0.1688 - acc: 0.9816 - val_loss: 1.3094 - val_acc: 0.7157

Epoch 00002: val_acc improved from 0.62500 to 0.71569, saving model to /data/oxford102/experiments/1544329584.4107325/mobilenet_v2_1544329584.4107325.h5
Epoch 3/100
17/17 [==============================] - 12s 686ms/step - loss: 0.1453 - acc: 0.9779 - val_loss: 1.2163 - val_acc: 0.6797

Epoch 00003: val_acc did not improve from 0.71569
Epoch 4/100
17/17 [==============================] - 12s 686ms/step - loss: 0.2289 - acc: 0.9596 - val_loss: 1.0969 - val_acc: 0.7344

Epoch 00004: val_acc improved from 0.71569 to 0.73438, saving model to /data/oxford102/experiments/15443295

In [15]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': model_name,
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
35/35 [==============================] - 28s 793ms/step - loss: 0.2139 - acc: 0.9625 - val_loss: 1.0995 - val_acc: 0.7109

Epoch 00001: val_acc improved from -inf to 0.71094, saving model to /data/oxford102/experiments/1544329740.684749/mobilenet_v2_1544329740.684749.h5
Epoch 2/100
35/35 [==============================] - 24s 680ms/step - loss: 0.1259 - acc: 0.9837 - val_loss: 1.0764 - val_acc: 0.7578

Epoch 00002: val_acc improved from 0.71094 to 0.75781, saving model to /data/oxford102/experiments/1544329740.684749/mobilenet_v2_1544329740.684749.h5
Epoch 3/100
35/35 [==============================] - 23s 671ms/step - loss: 0.1215 - acc: 0.9857 - val_loss: 1.1856 - val_acc: 0.6913

Epoch 00003: val_acc did not improve from 0.75781
Epoch 4/100
35/35 [==============================] - 24s 678ms/step - loss: 0.1274 - acc: 0.9813 - val_loss: 1.0628 - val_acc: 0.7422

Epoch 00004: val_acc did not improve from 0.75781
Epoch 5/100
35/35 [==============================] - 24s 681m

In [16]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': model_name,
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
71/71 [==============================] - 53s 753ms/step - loss: 0.1639 - acc: 0.9762 - val_loss: 1.1253 - val_acc: 0.7181

Epoch 00001: val_acc improved from -inf to 0.71815, saving model to /data/oxford102/experiments/1544329882.089569/mobilenet_v2_1544329882.089569.h5
Epoch 2/100
71/71 [==============================] - 49s 686ms/step - loss: 0.1255 - acc: 0.9815 - val_loss: 1.1536 - val_acc: 0.7261

Epoch 00002: val_acc improved from 0.71815 to 0.72610, saving model to /data/oxford102/experiments/1544329882.089569/mobilenet_v2_1544329882.089569.h5
Epoch 3/100
71/71 [==============================] - 48s 682ms/step - loss: 0.1108 - acc: 0.9824 - val_loss: 1.1970 - val_acc: 0.6931

Epoch 00003: val_acc did not improve from 0.72610
Epoch 4/100
71/71 [==============================] - 49s 685ms/step - loss: 0.1124 - acc: 0.9810 - val_loss: 1.1961 - val_acc: 0.7316

Epoch 00004: val_acc improved from 0.72610 to 0.73162, saving model to /data/oxford102/experiments/1544329882.0

In [17]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': model_name,
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
143/143 [==============================] - 102s 717ms/step - loss: 0.1109 - acc: 0.9803 - val_loss: 1.1367 - val_acc: 0.7241

Epoch 00001: val_acc improved from -inf to 0.72411, saving model to /data/oxford102/experiments/1544330253.19369/mobilenet_v2_1544330253.19369.h5
Epoch 2/100
143/143 [==============================] - 98s 683ms/step - loss: 0.0811 - acc: 0.9889 - val_loss: 1.2432 - val_acc: 0.7090

Epoch 00002: val_acc did not improve from 0.72411
Epoch 3/100
143/143 [==============================] - 98s 682ms/step - loss: 0.0959 - acc: 0.9847 - val_loss: 1.1647 - val_acc: 0.7260

Epoch 00003: val_acc improved from 0.72411 to 0.72599, saving model to /data/oxford102/experiments/1544330253.19369/mobilenet_v2_1544330253.19369.h5
Epoch 4/100
143/143 [==============================] - 98s 682ms/step - loss: 0.0768 - acc: 0.9886 - val_loss: 1.1317 - val_acc: 0.7250

Epoch 00004: val_acc did not improve from 0.72599
Epoch 5/100
143/143 [==============================] - 9

In [18]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': model_name,
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 16,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
287/287 [==============================] - 201s 699ms/step - loss: 0.0806 - acc: 0.9874 - val_loss: 1.1497 - val_acc: 0.7232

Epoch 00001: val_acc improved from -inf to 0.72316, saving model to /data/oxford102/experiments/1544330862.5871413/mobilenet_v2_1544330862.5871413.h5
Epoch 2/100
287/287 [==============================] - 195s 680ms/step - loss: 0.0666 - acc: 0.9901 - val_loss: 1.1626 - val_acc: 0.7218

Epoch 00002: val_acc did not improve from 0.72316
Epoch 3/100
287/287 [==============================] - 195s 679ms/step - loss: 0.0582 - acc: 0.9915 - val_loss: 1.1660 - val_acc: 0.7194

Epoch 00003: val_acc did not improve from 0.72316
Epoch 4/100
287/287 [==============================] - 195s 679ms/step - loss: 0.0541 - acc: 0.9918 - val_loss: 1.2133 - val_acc: 0.7232

Epoch 00004: val_acc improved from 0.72316 to 0.72316, saving model to /data/oxford102/experiments/1544330862.5871413/mobilenet_v2_1544330862.5871413.h5
Epoch 5/100
287/287 [========================

In [19]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "mobilenet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': model_name,
         'image_aug': False,
         'optimizer': 'RMSprop',
          'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
575/575 [==============================] - 397s 690ms/step - loss: 0.0552 - acc: 0.9925 - val_loss: 1.2356 - val_acc: 0.7090

Epoch 00001: val_acc improved from -inf to 0.70904, saving model to /data/oxford102/experiments/1544332252.6100016/mobilenet_v2_1544332252.6100016.h5
Epoch 2/100
575/575 [==============================] - 391s 681ms/step - loss: 0.0414 - acc: 0.9940 - val_loss: 1.2587 - val_acc: 0.7142

Epoch 00002: val_acc improved from 0.70904 to 0.71422, saving model to /data/oxford102/experiments/1544332252.6100016/mobilenet_v2_1544332252.6100016.h5
Epoch 3/100
459/575 [======================>.......] - ETA: 1:05 - loss: 0.0390 - acc: 0.9948{'network_name': 'mobilenet_v2', 'image_aug': False, 'optimizer': 'RMSprop', 'optimizer_params': None, 'batch_size': 8, 'epochs': 100, 'image_size': (256, 256), 'log_time': 1544332252.6100016}


KeyboardInterrupt: 

# FINETUNE TOP LAYERS

In [ ]:
for i, layer in enumerate(base_model.layers[619:]):
    print(i, layer.name)
    layer.trainable = True

In [12]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
71/71 [==============================] - 51s 723ms/step - loss: 0.1369 - acc: 0.9802 - val_loss: 1.1463 - val_acc: 0.6834

Epoch 00001: val_acc improved from -inf to 0.68340, saving model to /data/oxford102/experiments/1544327751.1324658/vgg19_1544327751.1324658.h5
Epoch 2/100
71/71 [==============================] - 47s 665ms/step - loss: 0.1201 - acc: 0.9859 - val_loss: 1.2191 - val_acc: 0.6985

Epoch 00002: val_acc improved from 0.68340 to 0.69853, saving model to /data/oxford102/experiments/1544327751.1324658/vgg19_1544327751.1324658.h5
Epoch 3/100
71/71 [==============================] - 47s 661ms/step - loss: 0.1182 - acc: 0.9824 - val_loss: 1.0393 - val_acc: 0.7124

Epoch 00003: val_acc improved from 0.69853 to 0.71236, saving model to /data/oxford102/experiments/1544327751.1324658/vgg19_1544327751.1324658.h5
Epoch 4/100
71/71 [==============================] - 47s 663ms/step - loss: 0.1097 - acc: 0.9832 - val_loss: 1.1543 - val_acc: 0.7096

Epoch 00004: val_acc did 